In [1]:

!pip install -q faiss-cpu sentence-transformers

import os, gc, pickle
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
import faiss
import torch
from sentence_transformers import SentenceTransformer

# -----------------------------
# CONFIG - UPDATE PATHS if needed
# -----------------------------
TRAIN_CSV = "/kaggle/input/amazon-ml-challenge-2025/student_resource/dataset/train.csv"
TEST_CSV  = "/kaggle/input/amazon-ml-challenge-2025/student_resource/dataset/test.csv"
TEXT_COL = "catalog_content"
COST_COL = "price"
SAMPLE_ID_COL = "sample_id"

EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
EMB_SAVE = "./embeddings.npy"
FAISS_INDEX_PATH = "./faiss_text_index_cpu.bin"
META_PATH = "./faiss_text_meta.pkl"

BATCH_SIZE = 256
TOP_K = 5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print("DEVICE for embeddings:", DEVICE)
print("faiss lib:", faiss.__name__, getattr(faiss, "__version__", "unknown"))

# -----------------------------
# 1) load train
# -----------------------------
train_df = pd.read_csv(TRAIN_CSV)
print("Train rows:", len(train_df))

texts = train_df[TEXT_COL].fillna("").astype(str).tolist()
costs = train_df[COST_COL].astype(float).tolist()

# -----------------------------
# 2) load embedder (on DEVICE)
# -----------------------------
print("Loading sentence-transformers model on", DEVICE)
model = SentenceTransformer(EMBED_MODEL, device=DEVICE)

# -----------------------------
# 3) compute or load embeddings (L2-normalized)
# -----------------------------
def compute_embeddings_if_missing(texts, save_path=EMB_SAVE, batch_size=BATCH_SIZE):
    if os.path.exists(save_path):
        print("Loading existing embeddings from", save_path)
        embs = np.load(save_path)
        return embs.astype(np.float32)
    print("Computing embeddings for", len(texts), "texts ...")
    embs_list = []
    for i in tqdm(range(0, len(texts), batch_size), desc="embedding"):
        batch = texts[i:i+batch_size]
        e = model.encode(batch, convert_to_numpy=True, normalize_embeddings=True, batch_size=batch_size)
        embs_list.append(e)
    embs = np.vstack(embs_list).astype(np.float32)
    np.save(save_path, embs)
    print("Saved embeddings to", save_path, "shape:", embs.shape)
    return embs

embeddings = compute_embeddings_if_missing(texts)

# -----------------------------
# 4) Build CPU FAISS index (IndexFlatIP)
# -----------------------------
def build_and_save_cpu_index(embs, index_path=FAISS_INDEX_PATH):
    d = embs.shape[1]
    print(f"Building CPU IndexFlatIP with dim={d}")
    cpu_index = faiss.IndexFlatIP(d)   # inner product; with normalized embs => cosine
    cpu_index.add(embs)
    print("CPU index ntotal:", cpu_index.ntotal)
    try:
        faiss.write_index(cpu_index, index_path)
        print("Wrote CPU index to", index_path)
    except Exception as e:
        print("Warning: failed saving index:", e)
    return cpu_index

# If index exists on disk, load it; else build
if os.path.exists(FAISS_INDEX_PATH):
    try:
        cpu_index = faiss.read_index(FAISS_INDEX_PATH)
        print("Loaded FAISS index from", FAISS_INDEX_PATH)
    except Exception as e:
        print("Failed to load existing index (will rebuild):", e)
        cpu_index = build_and_save_cpu_index(embeddings)
else:
    cpu_index = build_and_save_cpu_index(embeddings)

# Save metadata (costs, texts)
meta = {"costs": costs, "texts": texts}
with open(META_PATH, "wb") as f:
    pickle.dump(meta, f)
print("Saved metadata to", META_PATH)

# Optionally free embeddings to reduce RAM (keep if you want to re-run quickly)
# del embeddings
# gc.collect()
# if torch.cuda.is_available(): torch.cuda.empty_cache()

# -----------------------------
# 5) Prediction helper - batched
# -----------------------------
with open(META_PATH, "rb") as f:
    meta_loaded = pickle.load(f)
train_costs = np.array(meta_loaded["costs"], dtype=np.float32)

def predict_batch_texts(texts_batch, top_k=TOP_K, batch_size=BATCH_SIZE):
    # embed (on DEVICE)
    embs = model.encode(texts_batch, convert_to_numpy=True, normalize_embeddings=True, batch_size=batch_size).astype(np.float32)
    # search using CPU FAISS
    D, I = cpu_index.search(embs, top_k)
    preds = []
    for sims, inds in zip(D, I):
        sims = np.clip(sims, 0.0, None)
        costs_nn = train_costs[inds]
        if sims.sum() <= 0:
            preds.append(float(costs_nn.mean()))
        else:
            w = sims / (sims.sum() + 1e-12)
            preds.append(float((w * costs_nn).sum()))
    return np.array(preds, dtype=np.float32)

# -----------------------------
# 6) Run inference on test set (batched)
# -----------------------------
test_df = pd.read_csv(TEST_CSV)
print("Test rows:", len(test_df))

out_frames = []
for start in tqdm(range(0, len(test_df), BATCH_SIZE), desc="predicting"):
    batch = test_df.iloc[start:start+BATCH_SIZE]
    texts_batch = batch[TEXT_COL].fillna("").astype(str).tolist()
    preds = predict_batch_texts(texts_batch, top_k=TOP_K, batch_size=BATCH_SIZE)
    out_df = pd.DataFrame({
        SAMPLE_ID_COL: batch[SAMPLE_ID_COL].tolist(),
        "price": preds
    })
    out_frames.append(out_df)

final_preds_df = pd.concat(out_frames, ignore_index=True)
print("Predictions shape:", final_preds_df.shape)

# -----------------------------
# 7) Save submission
# -----------------------------
OUT_NAME = "submission.csv"
final_preds_df.to_csv(OUT_NAME, index=False)
print("Saved submission to", OUT_NAME)

# -----------------------------
# Done
# -----------------------------
print("Completed. Embeddings saved:", EMB_SAVE, "FAISS (CPU) saved:", FAISS_INDEX_PATH, "Meta:", META_PATH)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 52.7 MB/s eta 0:00:00:00:0100:01


2025-10-11 10:00:42.446496: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760176842.468698     233 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760176842.475650     233 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


DEVICE for embeddings: cuda
faiss lib: faiss 1.12.0
Train rows: 75000
Loading sentence-transformers model on cuda


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Loading existing embeddings from ./embeddings.npy
Building CPU IndexFlatIP with dim=384
CPU index ntotal: 75000
Wrote CPU index to ./faiss_text_index_cpu.bin
Saved metadata to ./faiss_text_meta.pkl
Test rows: 75000


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Predictions shape: (75000, 2)
Saved submission to submission.csv
Completed. Embeddings saved: ./embeddings.npy FAISS (CPU) saved: ./faiss_text_index_cpu.bin Meta: ./faiss_text_meta.pkl


In [5]:
len(final_preds)

NameError: name 'final_preds' is not defined